## Problem: Longest Increasing Subsequence
LeetCode : 300. Longest Increasing Subsequence

https://leetcode.com/problems/longest-increasing-subsequence/

Given an integer array nums, return the length of the longest strictly increasing 
subsequence
.

 

Example 1:

    Input: nums = [10,9,2,5,3,7,101,18]
    Output: 4
    Explanation: The longest increasing subsequence is [2,3,7,101], therefore the length is 4.
Example 2:

    Input: nums = [0,1,0,3,2,3]
    Output: 4
Example 3:

    Input: nums = [7,7,7,7,7,7,7]
    Output: 1
 

Constraints:

    1 <= nums.length <= 2500
    -104 <= nums[i] <= 104


### Approach:
To get the longest increasing subsequence, if current number is greater than previous number, then increase the count and recursively call for rest of the array.
Elase, compare the previous number with next number in the array.
Recursive method will take current Inex and previous index. Start with current Index = 0, and previous Index = -1.
Return the maximum count.

In [55]:
def longestIncreasingSubsequence_rec(nums):
    return LIS_rec(nums, 0, -1)

def LIS_rec(nums, cur, prev):
    if cur >= len(nums):return 0
    count = 0
    if prev == -1 or nums[cur]> nums[prev]:
        count =  1 + LIS_rec(nums, cur+1, cur)
    count = max(count, LIS_rec(nums, cur+1, prev))
    return count
    
        

In [56]:
nums = [10,9,2,5,3,7,101,18]
longestIncreasingSubsequence_rec(nums)

4

In [57]:
nums = [7,7,7,7,7,7,7]
longestIncreasingSubsequence_rec(nums)

1

## Memoization

In [58]:
def longestIncreasingSubsequence_mem(nums):
    n = len(nums)
    mem = [[-1]* n for _ in range(n)]
    return LIS_mem(nums, 0, -1, mem)

def LIS_mem(nums, cur, prev, mem):
    if cur >= len(nums):return 0
    if mem[cur][prev+1] != -1:
        return mem[cur][prev+1]
    count = 0
    if prev == -1 or nums[cur]> nums[prev]:
        count =  1 + LIS_mem(nums, cur+1, cur, mem)
    count = max(count, LIS_mem(nums, cur+1, prev, mem))
    mem[cur][prev+1] = count
    return mem[cur][prev+1]

In [59]:
nums = [10,9,2,5,3,7,101,18]
longestIncreasingSubsequence_mem(nums)

4

In [60]:
nums = [7,7,7,7,7,7,7]
longestIncreasingSubsequence_mem(nums)

1

## DP

The above algorithm tells us two things:

If the number at the currIndex is bigger than the number at the prevIndex, we increment the count for LIS up to the currIndex.

But if there is a bigger LIS without including the number at the currIndex, we take that. 

So we need to find all the increasing subsequences for the number at index i, from all the previous numbers (i.e. number until index i-1), to eventually find the longest increasing subsequence.

If i represents the currIndex and j represents the prevIndex, our recursive formula would look like:

    if num[i] > num[j] => dp[i] = dp[j] + 1 if there is no bigger LIS for 'i'

In [63]:
def longestIncreasingSubsequence_dp(nums):
    n = len(nums)
    dp = [1] * n
    maxLen = 1
    for i in range(n): #Current number
        for j in range(i): # Prev number
            if nums[i]> nums[j] and dp[i] <= dp[j]:
                dp[i] = dp[j] + 1
            maxLen = max(maxLen, dp[i])
    return maxLen
           
    
                

In [64]:
nums = [10,9,2,5,3,7,101,18]
longestIncreasingSubsequence_dp(nums)

4

## Binary Search : O(nlogn)
Let's construct the idea from following example.

Consider the example nums = [2, 6, 8, 3, 4, 5, 1], let's try to build the increasing subsequences starting with an empty one: sub1 = [].

Let pick the first element, sub1 = [2].

6 is greater than previous number, sub1 = [2, 6]

8 is greater than previous number, sub1 = [2, 6, 8]

3 is less than previous number, we can't extend the subsequence sub1, but we must keep 3 because in the future there may have the longest subsequence start with [2, 3], sub1 = [2, 6, 8], sub2 = [2, 3].

With 4, we can't extend sub1, but we can extend sub2, so sub1 = [2, 6, 8], sub2 = [2, 3, 4].

With 5, we can't extend sub1, but we can extend sub2, so sub1 = [2, 6, 8], sub2 = [2, 3, 4, 5].

With 1, we can't extend neighter sub1 nor sub2, but we need to keep 1, so sub1 = [2, 6, 8], sub2 = [2, 3, 4, 5], sub3 = [1].

Finally, length of longest increase subsequence = len(sub2) = 4.

In the above steps, we need to keep different sub arrays (sub1, sub2..., subk) which causes poor performance. 

But we notice that we can just keep one sub array, when new number x is not greater than the last element of the subsequence sub, we do binary search to find the smallest element >= x in sub, and replace with number x.

Let's run that example nums = [2, 6, 8, 3, 4, 5, 1] again:

Let pick the first element, sub = [2].

6 is greater than previous number, sub = [2, 6]

8 is greater than previous number, sub = [2, 6, 8]

3 is less than previous number, so we can't extend the subsequence sub. We need to find the smallest number >= 
3 in sub, it's 6. Then we overwrite it, now sub = [2, 3, 8].

4 is less than previous number, so we can't extend the subsequence sub. We overwrite 8 by 4, so sub = [2, 3, 4].

5 is greater than previous number, sub = [2, 3, 4, 5].

1 is less than previous number, so we can't extend the subsequence sub. We overwrite 2 by 1, so sub = [1, 3, 4, 5].

Finally, length of longest increase subsequence = len(sub) = 4.

In [71]:
## It will give you the corrrect length, but it wont give you the right set of increasing numbers.
## Look at Longest Increasing Subsequence print problem to print the correct set of numbers.
def longestIncreasingSubsequence_bs(nums):
    sub = []
    for n in nums:
        if len(sub)==0 or sub[-1]<n:
            sub.append(n)
        else:
            idx = binarySearch(sub, n) # find smallest number in sub which is greater than n
            if sub[idx] < n:
                sub[idx + 1] = n
            else:
                sub[idx] = n
    return len(sub)

def binarySearch(array, num):
    left, right = 0, len(array)-1
    while left<= right:
        mid = left + (right-left)//2
        if array[mid]>num:
            right = mid-1
        elif array[mid] < num:
            left = mid+1
        else:
            left = mid
            break
    return left
    

In [72]:
nums = [10,9,2,5,3,7,101,18]
longestIncreasingSubsequence_bs(nums)

4

In [73]:
nums = [7,7,7,7,7,7,7]
longestIncreasingSubsequence_bs(nums)

1

In [74]:
nums = [1,3,5,4,7]
longestIncreasingSubsequence_bs(nums)

4